In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


class FitnessTracker():
    def __init__(self):
        self.path = "data/fitness_data.csv"
        self.df = None

    def load_data(self):
        self.df = pd.read_csv(self.path)
        self.df.columns = self.df.columns.str.strip().str.lower()
    
        if "duration" not in self.df.columns:
            for col in self.df.columns:
                if "time" in col or "minute" in col or "duration" in col:
                    self.df.rename(columns={col: "duration"}, inplace=True)
                    break
    
        print("Data-set Loaded from:", self.path)
        print("Shape:", self.df.shape)
        print("Columns:", list(self.df.columns))

    def check(self):
        if self.df is None:
            print("Data-set not loaded! please load the data set first")
            return False
        return True

    def preview(self):
        if self.check():
            print(self.df.head())

    def handle_missing_data(self):
        if self.check():
            print("\nHandling Missing Data...")
    
            before = self.df.isnull().sum()
            print("\nMissing values before:")
            print(before)
    
            numeric_cols = ["duration", "calories burned"]
    
            for col in numeric_cols:
                if col in self.df.columns:
                    median_value = np.median(self.df[col].dropna())
                    self.df[col].fillna(median_value, inplace=True)
    
            after = self.df.isnull().sum()
            print("\nMissing values after:")
            print(after)
    
            print("Missing data handled using MEDIAN values")

    
    def data_analysis(self):
        if self.check():
            print("\n===== DATA ANALYSIS (NumPy & Pandas) =====")
    
            self.df["date"] = pd.to_datetime(self.df["date"])
    
            daily_avg_calories = np.mean(
                self.df.groupby("date")["calories burned"].sum()
            )
            print("Average Daily Calories Burned:", round(daily_avg_calories, 2))
    
            total_time_activity = self.df.groupby("activity type")["duration"].sum()
            print("\nTotal Time Spent Per Activity:")
            print(total_time_activity)
    
            weekly_trend = self.df.groupby(
                self.df["date"].dt.to_period("W")
            )["calories burned"].sum()
    
            print("\nWeekly Calories Burned Trend:")
            print(weekly_trend)


    def log_activity(self, activity_type, duration, calories):
        if not self.check():
            return

        if duration <= 0 or calories <= 0:
            print("Duration and Calories must be positive values")
            return

        new_row = {
            "date": pd.Timestamp.today().strftime("%Y-%m-%d"),
            "activity type": activity_type,
            "duration": duration,
            "calories burned": calories
        }

        self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index=True)
        print("New activity logged successfully")

    def calculate_metrics(self):
        if self.check():
            total_calories = np.sum(self.df["calories burned"])
            avg_duration = np.mean(self.df["duration"])
            activity_freq = self.df["activity type"].value_counts()
            print("Total Calories Burned:", total_calories)
            print("Average Duration:", avg_duration)
            print("Activity Frequency:")
            print(activity_freq)

    def filter_by_activity(self):
        if self.check():
            activity = input("Enter activity type: ")
            filtered = self.df[self.df["activity type"].str.lower() == activity.lower()]
            print(filtered)

    def bar_time_per_activity(self):
        if self.check():
            data = self.df.groupby("activity type")["duration"].sum()
            plt.figure(figsize=(8, 5))
            data.plot(kind="bar")
            plt.title("Time Spent on Each Activity")
            plt.xlabel("Activity Type")
            plt.ylabel("Duration (Minutes)")
            plt.tight_layout()
            plt.show()

    def line_calories_over_time(self):
        if self.check():
            self.df["date"] = pd.to_datetime(self.df["date"])
            daily = self.df.groupby("date")["calories burned"].sum()
            plt.figure(figsize=(10, 5))
            plt.plot(daily.index, daily.values)
            plt.title("Calories Burned Over Time")
            plt.xlabel("Date")
            plt.ylabel("Calories")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()

    def pie_activity_distribution(self):
        if self.check():
            data = self.df["activity type"].value_counts()
            plt.figure(figsize=(7, 7))
            plt.pie(data.values, labels=data.index, autopct="%1.1f%%")
            plt.title("Activity Distribution")
            plt.show()

    def heatmap_duration_calories(self):
        if self.check():
            corr = self.df[["duration", "calories burned"]].corr()
            sns.heatmap(corr, annot=True)
            plt.title("Duration vs Calories Burned")
            plt.show()

    def generate_report(self):
        if self.check():
            print("\n===== FITNESS SUMMARY REPORT =====")
            print("Total Activities:", len(self.df))
            print("Total Calories Burned:", self.df["calories burned"].sum())
            print("Average Duration:", round(self.df["duration"].mean(), 2))
            print("Most Frequent Activity:")
            print(self.df["activity type"].value_counts().head(1))


eda = FitnessTracker()

while True:
    print("\n====== Personal Fitness Tracker ======")
    print("1. Load Data")
    print("2. Preview Data")
    print("3. Handle Missing Data (Median)")
    print("4. Data Analysis (NumPy & Pandas)")
    print("5. Log New Activity")
    print("6. Calculate Metrics")
    print("7. Filter by Activity Type")
    print("8. Bar Chart: Time per Activity")
    print("9. Line Chart: Calories Over Time")
    print("10. Pie Chart: Activity Distribution")
    print("11. Heatmap: Duration vs Calories")
    print("12. Generate Report")
    print("0. Exit")

    choice = int(input("Enter your choice: "))

    if choice == 1:
        eda.load_data()
    elif choice == 2:
        eda.preview()
    elif choice == 3:
        eda.handle_missing_data()
    elif choice == 4:
        eda.data_analysis()
    elif choice == 5:
        act = input("Activity Type: ")
        dur = float(input("Duration (minutes): "))
        cal = float(input("Calories Burned: "))
        eda.log_activity(act, dur, cal)
    elif choice == 6:
        eda.calculate_metrics()
    elif choice == 7:
        eda.filter_by_activity()
    elif choice == 8:
        eda.bar_time_per_activity()
    elif choice == 9:
        eda.line_calories_over_time()
    elif choice == 10:
        eda.pie_activity_distribution()
    elif choice == 11:
        eda.heatmap_duration_calories()
    elif choice == 12:
        eda.generate_report()
    elif choice == 0:
        print("Exiting the program.........")
        break
    else:
        print("Invalid Choice")



====== Personal Fitness Tracker ======
1. Load Data
2. Preview Data
3. Handle Missing Data (Median)
4. Data Analysis (NumPy & Pandas)
5. Log New Activity
6. Calculate Metrics
7. Filter by Activity Type
8. Bar Chart: Time per Activity
9. Line Chart: Calories Over Time
10. Pie Chart: Activity Distribution
11. Heatmap: Duration vs Calories
12. Generate Report
0. Exit


Enter your choice:  0


Exiting the program.........
